In [ ]:
import os
import random
import cv2
from pathlib import Path
from imgaug import augmenters as iaa

# Define paths and classes
classes = ['cell phone', 'remote', 'knife', 'book', 'spoon', 
           'cup',  'scissors', 'fork', 'toothbrush', 'ball']
images_dir = "../Objects/"
output_dir = "dataset/"

# Create dataset structure
os.makedirs(output_dir, exist_ok=True)
for subset in ['train', 'val', 'test']:
    os.makedirs(f"{output_dir}/images/{subset}", exist_ok=True)
    os.makedirs(f"{output_dir}/labels/{subset}", exist_ok=True)

# Augmentation pipeline
augmentations = iaa.Sequential([
    iaa.Fliplr(0.5),  # Horizontal flip
    iaa.Affine(rotate=(-25, 25)),  # Rotation
    iaa.Affine(scale=(0.8, 1.2)),  # Scaling
    iaa.Multiply((0.8, 1.2)),  # Brightness adjustment
    iaa.AdditiveGaussianNoise(scale=(10, 20)),  # Add Gaussian noise
])

# Split ratios
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

def get_class_id(image_path):
    class_ids = {
        "../Objects/O1.jpg": 1,
        "../Objects/O2.jpg": 2,
        "../Objects/O3.jpg": 3,
        "../Objects/O4.jpg": 4,
        "../Objects/O5.jpg": 5,
        "../Objects/O6.jpg": 6,
        "../Objects/O7.jpg": 7,
        "../Objects/O8.jpg": 8,
        "../Objects/O9.jpg": 9,
        "../Objects/O10.jpg": 10
    }

    return class_ids[image_path]

# Function to augment and save images
def augment_and_save(image_path, class_id, output_dir, subset, num_augmentations=10):
    image = cv2.imread(str(image_path))
    bbox = [0.2, 0.2, 0.8, 0.8]  # Dummy bbox (normalized x_center, y_center, width, height)
    annotations = f"{class_id} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n"

    for i in range(num_augmentations):
        # Augment image
        augmented_image = augmentations(image=image)
        augmented_image_name = f"{image_path.stem}_{subset}_{i}.jpg"
        cv2.imwrite(f"{output_dir}/images/{subset}/{augmented_image_name}", augmented_image)

        # Save corresponding label
        label_path = f"{output_dir}/labels/{subset}/{augmented_image_name.replace('.jpg', '.txt')}"
        with open(label_path, 'w') as f:
            f.write(annotations)

# Load images and augment them
all_images = list(Path(images_dir).glob("*.jpg"))
random.shuffle(all_images)

# Split images into train, val, test
num_images = len(all_images)
train_images = all_images[:int(num_images * train_ratio)]
val_images = all_images[int(num_images * train_ratio):int(num_images * (train_ratio + val_ratio))]
test_images = all_images[int(num_images * (train_ratio + val_ratio)):]

# Augment images for each subset
for subset, subset_images in zip(['train', 'val', 'test'], [train_images, val_images, test_images]):
    for _, image_path in enumerate(subset_images):
        class_id = get_class_id(str(image_path))
        augment_and_save(image_path, class_id, output_dir, subset)

# Create data.yaml
yaml_content = f"""
path: {output_dir}
train: images/train
val: images/val
test: images/test

names:
"""
for i, cls in enumerate(classes):
    yaml_content += f"  {i}: {cls}\n"

with open(f"{output_dir}/data.yaml", 'w') as f:
    f.write(yaml_content)

print("Dataset with augmented images created successfully!")

KeyError: '../Objects/O2.jpg'

In [ ]:
from ultralytics import YOLO
from roboflow import Roboflow

rf = Roboflow(api_key="cP490yDsfM5wI6OSBxBA")
project = rf.workspace("object-recognition-olmwb").project("cp467-group-project")
version = project.version(2)
dataset = version.download("yolov11")

In [ ]:
model = YOLO('yolo11x.pt') 

model.train(data=f"{dataset.location}/data.yaml", epochs=10, imgsz=64, batch=4)